In [2]:
import gdspy

def centered_rectangle(center, length, width, layer, datatype):
    cx, cy = center
    p1 = (cx - (length / 2.0), cy - (width / 2.0))
    p2 = (cx + (length / 2.0), cy + (width / 2.0))
    return gdspy.Rectangle(p1, p2, layer=layer, datatype=datatype)

In [3]:
# length refers to distance along the x axis (horizontal)
# width refers to distance along the y axis (vertical)
# All distances in um

Nb_sti_length = 4.5e3
NbSi_twiddle_middle_gap_length = 2e3
Nb_stm_length = 1.5e3
NbSi_middle_sense_gap_length = 2e3
Nb_sto_length = 1.5e3

NbSi_overhang_length = 5e2

Nb_width = 8e3
NbSi_width = 7e3

total_NbSi_length = 2 * NbSi_overhang_length + NbSi_twiddle_middle_gap_length + NbSi_middle_sense_gap_length + Nb_sti_length + Nb_stm_length + Nb_sto_length

#guard_width = 20
guard_separation = 10

nbsiWindowInset = 10

chip_length = 15e3
chip_width = 10e3

print("Total Device Length: "+str(round(total_NbSi_length*1e-3, 3))+" mm")
print("Total Device Height: "+str(round(Nb_width*1e-3, 3))+" mm")

Total Device Length: 12.5 mm
Total Device Height: 8.0 mm


In [4]:
lib = gdspy.GdsLibrary()

cell = lib.new_cell('FIRST')

x_sti = NbSi_overhang_length-((total_NbSi_length-Nb_sti_length)/2)
x_sto = -NbSi_overhang_length+((total_NbSi_length-Nb_sto_length)/2)
x_stm = x_sti + ((Nb_sti_length+Nb_stm_length)/2) + NbSi_twiddle_middle_gap_length

nbsiStrip = centered_rectangle(center = (0,0), length=total_NbSi_length, width=NbSi_width, layer=1, datatype=0)
nbsiWindow = gdspy.offset([nbsiStrip], -nbsiWindowInset, join_first=True, layer=3)
sti = centered_rectangle(center = (x_sti,0), length=Nb_sti_length, width=Nb_width, layer=2, datatype=0)
sto = centered_rectangle(center = (x_sto,0), length=Nb_sto_length, width=Nb_width, layer=2, datatype=0)
stm = centered_rectangle(center = (x_stm,0), length=Nb_stm_length, width=Nb_width, layer=2, datatype=0)

cell.add(nbsiStrip)
cell.add(nbsiWindow)
cell.add(sti)
cell.add(sto)
cell.add(stm)

nstFused = gdspy.offset(cell.get_polygons(), guard_separation, join_first=True, layer=4)
deviceExtent = centered_rectangle(center = (0,0), length=chip_length, width=chip_width, layer=4, datatype=0)
frame = gdspy.boolean(deviceExtent, nstFused, "xor", layer = 4)
cell.add(frame)

# Save the library in a file called 'first.gds'.
lib.write_gds('nst_gdspy.gds')

# Display all cells using the internal viewer.
#gdspy.LayoutViewer()